# Circuit Analyzer Demo

This notebook demonstrates the Circuit Analyzer project, which uses **Modified Nodal Analysis (MNA)** to solve DC circuits.

## Features
- Component classes: `Node`, `Resistor`, `VoltageSource`, `CurrentSource`
- MNA solver using NumPy linear algebra
- Verification tools: power conservation, KCL validation
- NetworkX visualization

In [ ]:
# Import the circuit analyzer modules
from components import Node, Resistor, VoltageSource, CurrentSource
from circuit import Circuit
import examples

# For visualization
import matplotlib.pyplot as plt
%matplotlib inline

---
## 1. Voltage Divider

The classic voltage divider demonstrates the fundamental principle:

$$V_{out} = V_{in} \times \frac{R_2}{R_1 + R_2}$$

### Circuit Diagram
```
    Vs(Vin) --> [R1] --> Vout --> [R2] --> GND
```

In [ ]:
# Create voltage divider manually
gnd = Node("GND")
vin = Node("Vin")
vout = Node("Vout")

vs = VoltageSource(10.0, vin, gnd)  # 10V source
r1 = Resistor(1000, vin, vout)       # 1k ohm
r2 = Resistor(2000, vout, gnd)       # 2k ohm

circuit = Circuit()
circuit.add([vs, r1, r2])
circuit.solve(gnd)

# Verify against formula
v_theoretical = 10.0 * 2000 / (1000 + 2000)
print(f"Theoretical Vout: {v_theoretical:.4f} V")
print(f"Simulated Vout:   {vout.voltage:.4f} V")
print(f"\nVerification: {'PASS' if abs(vout.voltage - v_theoretical) < 1e-10 else 'FAIL'}")

In [ ]:
# Display all summaries
print(circuit.node_summary())
print()
print(circuit.component_summary())
print()
print(circuit.power_summary())
print()
print(circuit.kcl_summary())

In [ ]:
# Visualize the circuit
fig, ax = circuit.draw(figsize=(8, 6))
plt.show()

---
## 2. Wheatstone Bridge

The Wheatstone bridge is **balanced** when:

$$\frac{R_1}{R_3} = \frac{R_2}{R_4}$$

When balanced, no current flows through the bridge resistor (R5).

### Circuit Diagram
```
            Vin
           /   \
         R1     R2
         /       \
       Va --R5-- Vb
         \       /
         R3     R4
           \   /
           GND
```

In [ ]:
# Balanced Wheatstone Bridge (R1/R3 = R2/R4 = 1)
result = examples.wheatstone_balanced(verbose=False)

print("BALANCED WHEATSTONE BRIDGE")
print("=" * 40)
print(f"R1/R3 = {result['ratio1']:.4f}")
print(f"R2/R4 = {result['ratio2']:.4f}")
print(f"\nBridge voltage (Va - Vb): {result['bridge_voltage']:.6f} V")
print(f"Bridge current: {result['bridge_current']*1000:.6f} mA")
print(f"\nStatus: {'BALANCED' if result['is_balanced'] else 'UNBALANCED'}")

In [ ]:
# Visualize balanced bridge
fig, ax = result['circuit'].draw(figsize=(10, 8), layout='circular')
ax.set_title("Balanced Wheatstone Bridge", fontsize=14, fontweight='bold')
plt.show()

In [ ]:
# Unbalanced Wheatstone Bridge
result_unbal = examples.wheatstone_unbalanced(verbose=False)

print("UNBALANCED WHEATSTONE BRIDGE")
print("=" * 40)
print(f"R1/R3 = {result_unbal['ratio1']:.4f}")
print(f"R2/R4 = {result_unbal['ratio2']:.4f}")
print(f"\nBridge voltage (Va - Vb): {result_unbal['bridge_voltage']:.6f} V")
print(f"Bridge current: {result_unbal['bridge_current']*1000:.6f} mA")
print(f"\nStatus: {'BALANCED' if result_unbal['is_balanced'] else 'UNBALANCED'}")

---
## 3. Complex Network (5+ Nodes)

This example demonstrates the MNA solver's ability to handle larger circuits with multiple nodes and parallel paths.

In [ ]:
# Complex network example
result = examples.complex_network(verbose=False)

print(result['circuit'].node_summary())
print()
print(result['circuit'].power_summary())
print()
print(result['circuit'].kcl_summary())

In [ ]:
# Visualize complex network
fig, ax = result['circuit'].draw(figsize=(12, 10), layout='kamada_kawai')
ax.set_title("Complex Network (5+ Nodes)", fontsize=14, fontweight='bold')
plt.show()

---
## 4. Current Source Example

Demonstrating MNA with current sources instead of voltage sources.

In [ ]:
result = examples.current_source_example(verbose=False)

print("CURRENT SOURCE CIRCUIT")
print("=" * 40)
print(f"Theoretical V_N1: {result['theoretical']['n1']:.4f} V")
print(f"Simulated V_N1:   {result['nodes']['n1'].voltage:.4f} V")
print(f"\nTheoretical V_N2: {result['theoretical']['n2']:.4f} V")
print(f"Simulated V_N2:   {result['nodes']['n2'].voltage:.4f} V")
print(f"\nVerification: {'PASS' if result['verified'] else 'FAIL'}")

---
## 5. Mixed Sources

Circuit with both voltage and current sources, demonstrating the full MNA capability.

In [ ]:
result = examples.mixed_sources_example(verbose=False)

print(result['circuit'].node_summary())
print()
print(result['circuit'].component_summary())
print()
print(result['circuit'].power_summary())

In [ ]:
# Visualize mixed sources circuit
fig, ax = result['circuit'].draw(figsize=(8, 6))
ax.set_title("Mixed Sources Circuit", fontsize=14, fontweight='bold')
plt.show()

---
## 6. Build Your Own Circuit

Use the components to build and analyze your own circuit!

In [ ]:
# Example: Create a custom circuit
gnd = Node("GND")
n1 = Node("N1")
n2 = Node("N2")
n3 = Node("N3")

# Add your components here
vs = VoltageSource(5.0, n1, gnd)
r1 = Resistor(100, n1, n2)
r2 = Resistor(200, n2, n3)
r3 = Resistor(300, n3, gnd)
r4 = Resistor(150, n2, gnd)  # Parallel path

# Build and solve
my_circuit = Circuit()
my_circuit.add([vs, r1, r2, r3, r4])
my_circuit.solve(gnd)

# Display results
print(my_circuit.node_summary())
print()
print(my_circuit.power_summary())

In [ ]:
# Visualize your custom circuit
fig, ax = my_circuit.draw(figsize=(10, 8))
plt.show()

---
## Summary

This Circuit Analyzer demonstrates:

1. **Modified Nodal Analysis (MNA)** - The algorithm used by SPICE simulators
2. **Component Stamping** - How each component contributes to the system matrix
3. **Verification Methods** - Power conservation and KCL validation
4. **Visualization** - NetworkX topology graphs with node voltages and currents

### Key Equations

**MNA Matrix Structure:**
$$\begin{bmatrix} G & B \\ C & D \end{bmatrix} \begin{bmatrix} v \\ j \end{bmatrix} = \begin{bmatrix} i \\ e \end{bmatrix}$$

Where:
- **G** = Conductance matrix (from resistors)
- **B, C** = Voltage source connection matrices
- **v** = Unknown node voltages
- **j** = Unknown voltage source currents

**Verification:**
- Power: $P_{sources} = P_{resistors}$
- KCL: $\sum I_{node} = 0$ at each node